# Трекинг и подсчет пешеходов

Использованные библиотеки: ultralytics, supervision

In [1]:
import supervision as sv
from supervision.assets import download_assets, VideoAssets
import numpy as np
from ultralytics import YOLO
import torch

In [2]:
# Загрузка датасета с идущими людьми

video_path = download_assets(VideoAssets.PEOPLE_WALKING) 
sv.VideoInfo.from_video_path(video_path)

people-walking.mp4 asset download complete. 



VideoInfo(width=1920, height=1080, fps=25, total_frames=341)

In [3]:
# Создание модели YOLOv11 и перенос ее на GPU при возможности

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = YOLO("yolo11s.pt").to(device)
device

'cuda'

In [4]:
# получение идентификатора класса человек в модели YOLO

class_names = ['person']
id_to_name = model.model.names
name_to_id = {k: v for v, k in id_to_name.items()}
class_ids = [name_to_id[class_name] for class_name in class_names]
class_ids

[0]

In [5]:
# информация о видео
video_info = sv.VideoInfo.from_video_path(video_path)

# Создание ByteTracker для трекинга и подсчета
byte_tracker = sv.ByteTrack(
    track_activation_threshold=0.2, # порог для распознавания
    lost_track_buffer=75, # количество кадров для потери объекта
    minimum_matching_threshold=0.7, # порог для принятия второго распознавания того же объекта
    frame_rate=video_info.fps, # кадров/сек
    minimum_consecutive_frames=3) # минимальное количество кадров для распознавания

byte_tracker.reset()

# генератор кадров
generator = sv.get_video_frames_generator(video_path)

# Аннотаторы для аннотации объектов на видео
box_annotator = sv.BoxAnnotator(thickness=4)
label_annotator = sv.LabelAnnotator(text_thickness=2, text_scale=1.5, text_color=sv.Color.BLACK)
trace_annotator = sv.TraceAnnotator(thickness=4, trace_length=50)

# callback функция для трекинга и аннотации кадра
def callback(frame: np.ndarray, index: int) -> np.ndarray:
    # предсказание модели
    results = model(frame, verbose=False)[0]
    # перевод предсказания в формат supervision detections
    detections = sv.Detections.from_ultralytics(results)
    # выбираем распознавания только нужных классов
    detections = detections[np.isin(detections.class_id, class_ids)]
    # трекинг с помощью ByteTrack
    detections = byte_tracker.update_with_detections(detections)
    labels = [
        f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
        for confidence, class_id, tracker_id
        in zip(detections.confidence, detections.class_id, detections.tracker_id)
    ]

    # аннотация
    annotated_frame = frame.copy()
    annotated_frame = trace_annotator.annotate(
        scene=annotated_frame, detections=detections)
    annotated_frame = box_annotator.annotate(
        scene=annotated_frame, detections=detections)
    annotated_frame = label_annotator.annotate(
        scene=annotated_frame, detections=detections, labels=labels)

    return annotated_frame

# обработка видео
sv.process_video(
    source_path = video_path,
    target_path = 'result.mp4',
    callback=callback
)